In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


import imageio

from pandas_profiling import ProfileReport

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        

from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.over_sampling import SMOTE

DO_PCA = False
DO_PLOT = True
DO_CLASSIFY = True

df = pd.read_csv(file)
df.head()

def plot_confusion_matrix(test,pred):
    matrix = confusion_matrix(y_test, y_pred)
    plt.matshow(matrix)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('GroundTruth')
    plt.xlabel('Predicted')
    
    plt.show()

In [ ]:
df.info()

df.columns

In [ ]:
df['Social drinker'] = df['Social drinker'].astype('int')
df['Social smoker'] = df['Social smoker'].astype('int')
df['Disciplinary failure'] = df['Disciplinary failure'].astype('int')
df['Seasons'] = df['Seasons'].astype('category')
df['Education'] = df['Education'].astype('category')
df['Day of the week'] = df['Day of the week'].astype('category')
df['Month of absence'] = df['Month of absence'].astype('category')
df['Reason for absence'] = df['Reason for absence'].astype('category')
df["Work load Average/day"] = df["Work load Average/day "]

df = df.drop(["ID"], axis=1)

In [ ]:
def one_hot_encode(df, features):
    for feature in features:
        df = pd.concat([df,pd.get_dummies(df[feature], 
                                      prefix=feature)],axis=1).drop([feature],axis=1)
    return df

In [ ]:
df.isnull().sum()

In [ ]:
print(df['Month of absence'].value_counts())
df = df[df['Month of absence'] != 0]

In [ ]:
#df["is_weekend"] = df["Day of the week"].map (lambda val: 1 if val==6 else 0)

In [ ]:
#g = sns.FacetGrid(data=df,col='Day of the week')
#g.map(plt.hist,'Absenteeism time in hours')

In [ ]:
#if DO_PLOT:
#    sns.pairplot(df.select_dtypes(exclude='bool'), vars=["Absenteeism time in hours", "Month of absence", "Reason for absence"], height=4, aspect=1)

In [ ]:
#if DO_PLOT:
#    with sns.axes_style("white"):
#        sns.jointplot(x="Absenteeism time in hours", y="Reason for absence", data=df);

In [ ]:
#if DO_PLOT:
#    cat_cols = df.select_dtypes(include='category').columns
#    fig, axes = plt.subplots(1, len(cat_cols))

#    for i, col in enumerate(cat_cols):
#        sns.countplot(y=col, orient="h", data=df, ax=axes[i])

In [ ]:
#profile = ProfileReport(df, title="Pandas Profiling Report")
#profile.to_notebook_iframe()

In [ ]:
#profile.to_file("report.html")

In [ ]:
if DO_PLOT:

    age_abs = df.groupby('Age')[['Absenteeism time in hours']].mean()
    ax = age_abs.plot(kind='bar', figsize=(8,6), legend=False)
    for i, v in enumerate(age_abs.values):
        ax.text(i-.25, v + 1, str(np.int(np.round(v))), color='red')
    ax.set_ylabel('Absenteeism time in hours')
    #ax.set_title('Average Absenteeism time in hours by age')
    plt.show()

In [ ]:
if DO_PLOT:

    dis_abs = df.groupby('Distance from Residence to Work')[['Absenteeism time in hours']].mean()
    ax = dis_abs.plot(kind='bar', figsize=(8,6), legend=False)
    for i, v in enumerate(dis_abs.values):
        ax.text(i-.25, v + 1, str(np.int(np.round(v))), color='red')
    ax.set_xlabel('Distance from Residence to Work (km)')
    ax.set_ylabel('Absenteeism time in hours')
    #ax.set_title('Average Absenteeism time in hours by distance')
    plt.show()

In [ ]:
if DO_PLOT:
    fig, ax = plt.subplots(figsize=(11,11))
    sns.heatmap(df.corr(), square = True, ax=ax)

In [ ]:
my_dpi=96
plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
 
pca = PCA(n_components=3)
pca.fit(df)
 
# Store results of PCA in a data frame
result = pd.DataFrame(pca.transform(df), columns=['PCA%i' % i for i in range(3)], index=df.index)

angle = 70
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(result['PCA0'], result['PCA1'], result['PCA2'], cmap="Set2_r", s=60)

# make simple, bare axis lines through space:
xAxisLine = ((min(result['PCA0']), max(result['PCA0'])), (0, 0), (0,0))
ax.plot(xAxisLine[0], xAxisLine[1], xAxisLine[2], 'r')
yAxisLine = ((0, 0), (min(result['PCA1']), max(result['PCA1'])), (0,0))
ax.plot(yAxisLine[0], yAxisLine[1], yAxisLine[2], 'r')
zAxisLine = ((0, 0), (0,0), (min(result['PCA2']), max(result['PCA2'])))
ax.plot(zAxisLine[0], zAxisLine[1], zAxisLine[2], 'r')

ax.view_init(30,angle)

# label the axes
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
#ax.set_title("PCA")
plt.show()

In [ ]:
def plot_pca_expvar(pca):
    """Plot the variance explained by the given PCA model 
    as a function of the number of principal components.
    """
    plt.figure(figsize=(8, 4))
    variance_ratio = pca.explained_variance_ratio_
    xticks = np.arange(pca.n_components_) + 1
    #plt.title('Explained variance')
    plt.grid(True)
    plt.xticks(xticks)
    plt.ylim(0, 1)
    plt.ylabel('Ratio')
    plt.plot(xticks, variance_ratio, label='Explained variance')
    plt.plot(xticks, np.cumsum(variance_ratio), label='Cumulative')
    plt.legend()

In [ ]:
pca = PCA(n_components=3)
plot_pca_expvar(pca.fit(df))

In [ ]:
df = one_hot_encode(df, ["Reason for absence", "Seasons", "Month of absence", "Day of the week", "Education"])

In [ ]:
to_normalize = ["Transportation expense", "Distance from Residence to Work", 
                "Service time", "Work load Average/day", "Weight", "Height", "Body mass index"]


for column in to_normalize:
    x = df[column].values 
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x.reshape(-1,1))
    df[column] = x_scaled

In [ ]:
#bins = [25, 35, 45, 55, np.inf]
#names = [25, 35, 45, 55]
#df['Age'] = pd.cut(df['Age'], bins, labels=names)

In [ ]:
if DO_CLASSIFY:
    
        abs_bins = [-1, 0, 10, np.inf]
        abs_names = ['0', '0-10', '>10']
        
        df["Absenteeism time in hours"] = pd.cut(df["Absenteeism time in hours"], abs_bins ,include_lowest=True, labels=abs_names)
df["Absenteeism time in hours"].value_counts()

In [ ]:
y = df["Absenteeism time in hours"]
if not DO_PCA:
    X = df.drop(["Absenteeism time in hours"], axis=1)

else:
    pca = PCA(n_components=3)
    X = pca.fit_transform(df.drop(["Absenteeism time in hours"], axis=1))


In [ ]:
if DO_CLASSIFY:
    smote = SMOTE(sampling_strategy='auto',random_state=0)
    X,y = smote.fit_sample(X,y)

In [ ]:
X.isnull().sum()

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    #axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    #axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    #axes[2].set_title("Performance of the model")

    return plt

In [ ]:
from sklearn.model_selection import cross_val_predict

if DO_CLASSIFY:

    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    scores = cross_val_score(clf, X, y, cv=3)

    plt.plot(range(1,4),scores)
    plt.xticks(range(1,4))
    plt.grid()
    

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
plot_confusion_matrix(y_test, y_pred)

In [ ]:
if DO_CLASSIFY:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf.fit(X_train, y_train)
    print("Accuracy: ", accuracy_score(y_test, clf.predict(X_test)))

    ax = (pd.Series(clf.feature_importances_, index=X.columns)
           .nlargest(19)
           .plot(kind='barh', figsize=(8,6), color='lightgreen'))
    plt.show()

In [ ]:
if DO_CLASSIFY:
    clf.fit(X,y)

    fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
    plot_tree(clf.estimators_[0], feature_names = X.columns, filled=True)
    fig.savefig('imagename.png')

In [ ]:
if DO_CLASSIFY:
    svc = SVC(C=1.0,kernel='rbf')
    scores = cross_val_score(svc, X, y, cv=3)
    plt.plot(range(1,4),scores)
    plt.xticks(range(1,4))
    plt.grid()
    plot_learning_curve(svc, "title", X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))

In [ ]:
from sklearn.naive_bayes import GaussianNB

if DO_CLASSIFY:
    nb = GaussianNB(priors=None)
    scores = cross_val_score(nb, X, y, cv=3)
    plt.plot(range(1,4),scores)
    plt.xticks(range(1,4))
    plt.grid()
    plot_learning_curve(nb, "NB", X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
if DO_CLASSIFY:
    knn  = KNeighborsClassifier(n_neighbors=3)
    scores = cross_val_score(knn, X, y, cv=3)
    plt.plot(range(1,4),scores)
    plt.xticks(range(1,4))
    plt.grid()
    plot_learning_curve(knn, "KNN", X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))

In [ ]:
from sklearn.linear_model import LogisticRegression
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

if DO_CLASSIFY:
    lr  = LogisticRegression(random_state=0, max_iter=1000)
    scores = cross_val_score(lr, X, y, cv=3)
    plt.plot(range(1,4),scores)
    plt.xticks(range(1,4))
    plt.grid()
    plot_learning_curve(lr, "LR", X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

if not DO_CLASSIFY:
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = SVR(C=1.0)
    clf.fit(X_train,y_train)
    
    y_test = clf.predict(X_test)
    print("MSE: ", mean_squared_error(y_test, y_pred))
    print("R2 Score: ", r2_score(y_test, y_pred))
    

In [ ]:
if not DO_CLASSIFY:
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    reg = GradientBoostingRegressor(random_state=0)
    scores = cross_val_score(reg, X, y, cv=5)
    print(scores)

    reg.fit(X_train, y_train)
    print(r2_score(y_test, reg.predict(X_test)))

In [ ]:
if not DO_CLASSIFY:
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    reg = RandomForestRegressor(n_estimators=100, random_state=0)
    scores = cross_val_score(reg, X, y, cv=5)
    print(scores)

    reg.fit(X_train, y_train)
    print(r2_score(y_test, reg.predict(X_test)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import  VotingClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
lr  = LogisticRegression(random_state=0, max_iter=1000)
lda =  LinearDiscriminantAnalysis()

eclf1 = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('lda', lda)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
accuracy_score(y_test,eclf1.predict(X_test))

In [ ]:
plot_learning_curve(eclf1, "ensemble", X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))